# 2.2: Project Planning and Sourcing Data with an API

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

In [2]:
# Create a list with all files in the folder using a list comprehension

folderpath = r"Data" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
print(len(filepaths))

12


In [4]:
filepaths

['Data\\202201-citibike-tripdata_1.csv',
 'Data\\202202-citibike-tripdata_1.csv',
 'Data\\202203-citibike-tripdata_1.csv',
 'Data\\202204-citibike-tripdata_1.csv',
 'Data\\202205-citibike-tripdata_1.csv',
 'Data\\202206-citibike-tripdata_1.csv',
 'Data\\202207-citibike-tripdata_1.csv',
 'Data\\202208-citibike-tripdata_1.csv',
 'Data\\202209-citibike-tripdata_1.csv',
 'Data\\202210-citibike-tripdata_1.csv',
 'Data\\202211-citibike-tripdata_1.csv',
 'Data\\202212-citibike-tripdata_1.csv']

In [5]:
# Read and concatenate all files simultaneously

df = pd.concat((pd.read_csv(f, low_memory=False) for f in filepaths), ignore_index = True)

In [6]:
df.shape

(12000000, 13)

In [7]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member


In [8]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
11999995,2C18D66E198EA410,classic_bike,2022-12-04 20:11:00.788,2022-12-04 20:19:15.088,6 Ave & W 33 St,6364.07,9 Ave & W 39 St,6644.08,40.749013,-73.988484,40.756404,-73.994101,member
11999996,3C0686D88E2F6AB5,classic_bike,2022-12-14 18:51:45.520,2022-12-14 18:57:44.283,E 20 St & Park Ave,6055.08,Ave A & E 14 St,5779.11,40.738274,-73.987520,40.730311,-73.980472,member
11999997,42AAD1775F1D3138,classic_bike,2022-12-31 18:13:19.297,2022-12-31 18:19:11.314,E 20 St & Park Ave,6055.08,Ave A & E 14 St,5779.11,40.738274,-73.987520,40.730311,-73.980472,member
11999998,DA4C5D66E1C60D25,classic_bike,2022-12-04 23:51:05.474,2022-12-05 00:20:25.181,Vesey St & Church St,5216.06,W 45 St & 8 Ave,6676.02,40.712220,-74.010472,40.759291,-73.988597,member
11999999,A570DB28B3868263,electric_bike,2022-12-23 17:46:18.594,2022-12-23 17:51:21.043,E 20 St & Park Ave,6055.08,E 11 St & 1 Ave,5746.14,40.738039,-73.987402,40.729538,-73.984267,member


This code first defines the folder path ("Data") and uses a list comprehension to collect the full file paths of all files inside that folder. Then it reads each file into a pandas DataFrame with pd.read_csv and concatenates them all together into one large DataFrame using pd.concat. The argument ignore_index=True ensures that the row indices are reset in the combined DataFrame. Finally, df.shape shows the dimensions of the resulting dataset, while df.head() and df.tail() display the first and last few rows for quick inspection.

# Get weather data using NOAA's API

In [9]:
# Define private NOAA token (received from https://www.ncdc.noaa.gov/cdo-web/token by e-mail)

Token = 'PCwIkNDEhLvOKDkhZAKzfUIlMHVPlcEh'

In [10]:
import requests

# WINTER (January–February + December 2022)
url_winter_1 = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"   # mean daily temperature
    "&datatypeid=TMAX"   # max daily temperature
    "&datatypeid=TMIN"   # min daily temperature
    "&datatypeid=PRCP"   # daily precipitation
    "&datatypeid=AWND"   # average wind speed
    "&startdate=2022-01-01"
    "&enddate=2022-02-28"
    "&limit=1000"
)
r_winter_1 = requests.get(url_winter_1, headers={"token": Token})
print("Winter part 1 (Jan–Feb):", r_winter_1.status_code)

url_winter_2 = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-12-01"
    "&enddate=2022-12-31"
    "&limit=1000"
)
r_winter_2 = requests.get(url_winter_2, headers={"token": Token})
print("Winter part 2 (Dec):", r_winter_2.status_code)


# SPRING (March–May 2022)
url_spring = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-03-01"
    "&enddate=2022-05-31"
    "&limit=1000")
r_spring = requests.get(url_spring, headers={"token": Token})
print("Spring (Mar–May):", r_spring.status_code)


# SUMMER (June–August 2022)
url_summer = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-06-01"
    "&enddate=2022-08-31"
    "&limit=1000")

r_summer = requests.get(url_summer, headers={"token": Token})
print("Summer (Jun–Aug):", r_summer.status_code)


# FALL (September–November 2022)
url_fall = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-09-01"
    "&enddate=2022-11-30"
    "&limit=1000") 

r_fall = requests.get(url_fall, headers={"token": Token})
print("Fall (Sep–Nov):", r_fall.status_code)

Winter part 1 (Jan–Feb): 200
Winter part 2 (Dec): 200
Spring (Mar–May): 200
Summer (Jun–Aug): 200
Fall (Sep–Nov): 200


In [15]:
# Get the API 
#r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [11]:
# Combine all JSON results into one list
all_results = (
    r_winter_1.json()['results'] +
    r_winter_2.json()['results'] +
    r_spring.json()['results'] +
    r_summer.json()['results'] +
    r_fall.json()['results'])

# Convert to a DataFrame
weather_raw = pd.DataFrame(all_results)

# Check structure
print("Total records collected:", len(weather_raw))
print(weather_raw.head())

Total records collected: 1825
                  date datatype            station attributes  value
0  2022-01-01T00:00:00     AWND  GHCND:USW00014732       ,,W,     28
1  2022-01-01T00:00:00     PRCP  GHCND:USW00014732   ,,W,2400    193
2  2022-01-01T00:00:00     TAVG  GHCND:USW00014732      H,,S,    116
3  2022-01-01T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400    139
4  2022-01-01T00:00:00     TMIN  GHCND:USW00014732   ,,W,2400    100


In [12]:
# Pivot the raw table: each datatype becomes a column
weather_df = weather_raw.pivot(index='date', columns='datatype', values='value').reset_index()

# Rename columns for clarity
weather_df.rename(columns={
    'TAVG': 'avgTemp_tenthsC',
    'TMAX': 'temp_max_tenthsC',
    'TMIN': 'temp_min_tenthsC',
    'PRCP': 'total_precip_tenthsMM',
    'AWND': 'wind_speed_tenthsMS'
}, inplace=True)

# Convert NOAA "tenths" units to normal numeric units
weather_df['avgTemp'] = weather_df['avgTemp_tenthsC'] / 10     # °C
weather_df['temp_max'] = weather_df['temp_max_tenthsC'] / 10   # °C
weather_df['temp_min'] = weather_df['temp_min_tenthsC'] / 10   # °C
weather_df['total_precipitation'] = weather_df['total_precip_tenthsMM'] / 10  # mm
weather_df['wind_speed'] = weather_df['wind_speed_tenthsMS'] / 10             # m/s

# Keep only the useful columns
weather_df = weather_df[['date', 'avgTemp', 'temp_max', 'temp_min', 'total_precipitation', 'wind_speed']]

# Convert date to datetime and sort
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df = weather_df.sort_values('date').reset_index(drop=True)

# Check result
print("Weather DataFrame created:", weather_df.shape)
print(weather_df.head())

Weather DataFrame created: (365, 6)
datatype       date  avgTemp  temp_max  temp_min  total_precipitation  \
0        2022-01-01     11.6      13.9      10.0                 19.3   
1        2022-01-02     11.4      15.6       3.9                  1.0   
2        2022-01-03      1.4       3.9      -4.3                  0.0   
3        2022-01-04     -2.7       2.2      -6.0                  0.0   
4        2022-01-05      3.2       8.9       0.0                  6.1   

datatype  wind_speed  
0                2.8  
1                4.3  
2                6.4  
3                3.9  
4                3.4  


In [13]:
weather_df.shape

(365, 6)

In [14]:
weather_df.head()

datatype,date,avgTemp,temp_max,temp_min,total_precipitation,wind_speed
0,2022-01-01,11.6,13.9,10.0,19.3,2.8
1,2022-01-02,11.4,15.6,3.9,1.0,4.3
2,2022-01-03,1.4,3.9,-4.3,0.0,6.4
3,2022-01-04,-2.7,2.2,-6.0,0.0,3.9
4,2022-01-05,3.2,8.9,0.0,6.1,3.4


In [15]:
weather_df.columns.name = None

In [16]:
weather_df.head()

,date,avgTemp,temp_max,temp_min,total_precipitation,wind_speed
0,2022-01-01,11.6,13.9,10.0,19.3,2.8
1,2022-01-02,11.4,15.6,3.9,1.0,4.3
2,2022-01-03,1.4,3.9,-4.3,0.0,6.4
3,2022-01-04,-2.7,2.2,-6.0,0.0,3.9
4,2022-01-05,3.2,8.9,0.0,6.1,3.4


In [17]:
# Save the cleaned weather data to csv
weather_df.to_csv('LaGuardia_Weather_NY_2022.csv', index=False)

In [16]:
# Load the api response as a json

#d = json.loads(r.text)

In [17]:
#d

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

In [18]:
# Secure all items in the response that correspond to TAVG

#avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [19]:
# Get only the date field from all average temperature readings

#dates_temp = [item['date'] for item in avg_temps]

In [20]:
# Get the temperature from all average temperature readings

#temps = [item['value'] for item in avg_temps]

In [21]:
#temps

[116,
 114,
 14,
 -27,
 32,
 49,
 7,
 -25,
 14,
 16,
 -54,
 -19,
 40,
 48,
 -67,
 -80,
 39,
 18,
 32,
 51,
 -60,
 -59,
 -7,
 -2,
 36,
 -23,
 -42,
 1,
 -48,
 -71,
 -34,
 -17,
 23,
 64,
 58,
 -28,
 -48,
 5,
 41,
 28,
 63,
 86,
 118,
 28,
 -43,
 -47,
 16,
 116,
 99,
 1,
 -26,
 41,
 56,
 144,
 11,
 8,
 -11,
 24,
 8,
 26,
 77,
 56,
 -19,
 31,
 96,
 182,
 87,
 32,
 40,
 72,
 51,
 -23,
 53,
 116,
 136,
 82,
 139,
 131,
 143,
 104,
 107,
 72,
 57,
 96,
 104,
 59,
 -15,
 -12,
 26,
 99,
 131,
 72,
 70,
 76,
 97,
 86,
 91,
 128,
 114,
 86,
 88,
 134,
 154,
 163,
 157,
 157,
 83,
 79,
 76,
 94,
 106,
 146,
 133,
 128,
 104,
 115,
 116,
 82,
 104,
 129,
 136,
 121,
 124,
 120,
 172,
 148,
 100,
 104,
 135,
 154,
 161,
 171,
 160,
 179,
 188,
 204,
 197,
 178,
 155,
 154,
 212,
 278,
 210,
 174,
 162,
 164,
 202,
 204,
 207,
 233,
 283,
 178,
 203,
 195,
 219,
 200,
 222,
 218,
 237,
 244,
 218,
 218,
 205,
 242,
 239,
 233,
 203,
 252,
 217,
 173,
 205,
 221,
 191,
 192,
 219,
 258,
 270,
 243,
 22

In [22]:
# Put the results in a dataframe

#df_temp = pd.DataFrame()

In [23]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

#df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
#df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [24]:
#df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [25]:
#df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


## Merging bike trips and weather data

In [18]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [27]:
# ensuring the columns matching the data sets are in the same format

cat_cols = [
    'rideable_type',
    'start_station_name',
    'start_station_id',
    'end_station_name',
    'end_station_id',
    'member_casual'
]

for col in cat_cols:
    df[col] = df[col].astype('category')

In [21]:
float_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')

In [28]:
# convert the started_at column to datetime format
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [24]:
df.dtypes

ride_id                       object
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name          category
start_station_id            category
end_station_name            category
end_station_id              category
start_lat                    float32
start_lng                    float32
end_lat                      float32
end_lng                      float32
member_casual               category
dtype: object

In [29]:
# Extract only the date part (YYYY-MM-DD) from 'started_at' into a new column

df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [30]:
# Convert 'date' from string to datetime

df['date'] = pd.to_datetime(df['date'])

In [31]:
df.dtypes

ride_id                       object
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name          category
start_station_id            category
end_station_name            category
end_station_id              category
start_lat                    float32
start_lng                    float32
end_lat                      float32
end_lng                      float32
member_casual               category
date                  datetime64[ns]
dtype: object

In [32]:
# Merge dataframes

df_merged = df.merge(weather_df, how = 'left', on = 'date', indicator = True)

In [33]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,temp_max,temp_min,total_precipitation,wind_speed,_merge
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802116,-73.968178,40.804039,-73.945923,member,2022-01-21,-6.0,-4.9,-9.3,0.0,6.2,both
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673744,-73.985649,40.688488,-73.991158,member,2022-01-10,1.6,5.6,-3.2,0.0,7.5,both
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960938,40.745167,-73.986832,member,2022-01-26,-2.3,-1.0,-6.0,0.0,5.6,both
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783962,-73.947166,40.745167,-73.986832,member,2022-01-03,1.4,3.9,-4.3,0.0,6.4,both
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749641,-73.988052,40.745167,-73.986832,member,2022-01-22,-5.9,-1.0,-9.9,0.0,3.4,both


In [34]:
df_merged['_merge'].value_counts(dropna = False)

_merge
both          11999379
left_only          621
right_only           0
Name: count, dtype: int64

In [35]:
df_merged.shape

(12000000, 20)

In [36]:
df_merged.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgTemp', 'temp_max', 'temp_min',
       'total_precipitation', 'wind_speed', '_merge'],
      dtype='object')

In [37]:
# Save the merged df to pickle
df_merged.to_pickle('NY_data_merged.pkl')